In [987]:
import warnings
import pandas 
pandas.set_option('mode.use_inf_as_na', True)

warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

# This is required to catch warnings when the multiprocessing module is used
import os

os.environ["PYTHONWARNINGS"] = "ignore"
# import pertpy as pt
import scanpy as sc

import pertpy as pt
adata = pt.dt.kang_2018()
from torch import tensor
import torch, numpy as np, pandas as pd
np.set_printoptions(linewidth=140)
torch.set_printoptions(linewidth=140, sci_mode=False, edgeitems=7)
pd.set_option('display.width', 140)
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

In [1005]:
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata)

#hv_genes = filtered_keys = df[df['column_name'] == True].index.tolist()

#normal_genes = filtered_keys = df[df['column_name'] == False].index.tolist()

#print(hv_genes, normal_genes)
hv_genes = (list(adata.var[adata.var['highly_variable'] == True].index))
normal_genes = (list(adata.var_names))

hv_columns = [i for i,val in enumerate(normal_genes) if val in hv_genes]
#hv_genes
len(hv_columns)

In [991]:
#preprocess
#batchcontrol
#clustering
#cell type annotation
#embedding
#reverse transcriptome
#differential expression

#rna velocity -> predicts the future state 
#predict the probability of unseen perturbations
#test: first data set => output list of perturbations 
# take a 2nd dataset and get list of perturbations + their probability 


#representation learning (in particular, self-supervised, multi-view, and transfer learning
#https://registry.opendata.aws/tabula-muris/#usageexamples

#see what preprocessing is needed to get better accuracy

##load data

##import premade model => output list of predicted unseen perturbation
sc.pp.calculate_qc_metrics(adata)

#write custom model. get better accuracy

In [74]:
# gene_expressed = ["string1", "string2", "string3"]

    
# strings = np.array(gene_expressed)
# #adata.obs['genes_expressed'] = strings
# coordinates = []
cx = adata.X
cx = cx.tocoo()

# #delete non highly variable genes from matrix


# for i,j,v in zip(cx.row, cx.col, cx.data):
#     if i in hv_columns: coordinates += [[i,j,v]]
# for coord in coordinates:
#     coord[1] = adata.var_names[coord[1]]

#batch removal - one patient at at time
#one cell_type at a time 
#index cell type
#index ctrl/stim

In [1012]:
from collections import defaultdict
rowGeneExpression = defaultdict(int)
rowGeneExpression2 = defaultdict(dict)


import math
math.floor
df = adata.obs
end = 50 if True else -1
for column in hv_columns:
    for row_id in range(math.floor(float(df.shape[0])))[:end]:
        rowGeneExpression[row_id] += adata.X[row_id, column]
        #rowGeneExpression2[row_id][column] = adata.X[row_id, column]


[35.94687108695507,
 33.196212351322174,
 7.205496981739998,
 35.48313131928444,
 26.742908969521523,
 11.36419740319252,
 24.907880529761314,
 10.883503705263138,
 29.542999163269997,
 11.383434176445007,
 10.421529799699783,
 15.752853259444237,
 10.594988703727722,
 8.035826236009598,
 18.963955223560333,
 10.628947839140892,
 32.24674420058727,
 15.521971955895424,
 12.094075202941895,
 8.829016640782356,
 24.186710730195045,
 14.787614285945892,
 11.188271582126617,
 11.223772212862968,
 9.65641550719738,
 15.611911222338676,
 11.778271540999413,
 19.134468212723732,
 12.935988828539848,
 11.119527280330658,
 20.593149721622467,
 12.315789371728897,
 37.85372845828533,
 13.421977430582047,
 10.79379317164421,
 10.98905485868454,
 11.481004521250725,
 9.30767348408699,
 16.307302594184875,
 10.960439682006836,
 36.655866891145706,
 22.592174723744392,
 23.983603835105896,
 12.39954560995102,
 15.422866806387901,
 18.242857322096825,
 10.670321449637413,
 40.28811080753803,
 10.8072

In [1105]:
df = adata.obs
dependent_variables =  [rowGeneExpression[row] for row in range(df.shape[0])]
df['geneExpressionCount'] = dependent_variables
numerical_values = df.select_dtypes(include=[int, float]).values.tolist()
dependent_variables = [x for x in dependent_variables]


In [1106]:
criterion = torch.nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [1107]:
t_dep = tensor(dependent_variables) # pertrubations
t_dep

tensor([35.9469, 33.1962,  7.2055, 35.4831, 26.7429, 11.3642, 24.9079,  ...,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000], dtype=torch.float64)

In [1108]:
t_indep = tensor(numerical_values, dtype=torch.float)
t_indep

tensor([[  3017.0000,    877.0000,    -27.6404,     14.9666,      9.0000,   1704.0000,    711.0000,     35.9469],
        [  2481.0000,    713.0000,    -27.4936,     28.9249,      9.0000,   1614.0000,    662.0000,     33.1962],
        [   703.0000,    337.0000,    -10.4682,     -5.9844,      3.0000,    908.0000,    337.0000,      7.2055],
        [  3420.0000,    850.0000,    -24.3680,     20.4293,      9.0000,   1738.0000,    653.0000,     35.4831],
        [  3158.0000,   1111.0000,     27.9522,     24.1597,      4.0000,   1857.0000,    928.0000,     26.7429],
        [  1869.0000,    635.0000,     -0.4702,    -25.3987,      5.0000,   1525.0000,    634.0000,     11.3642],
        [  1142.0000,    436.0000,    -15.9062,     20.0853,      9.0000,   1157.0000,    436.0000,     24.9079],
        ...,
        [   635.0000,    424.0000,     -6.6479,     -5.5475,      3.0000,    882.0000,    423.0000,      0.0000],
        [  1340.0000,    480.0000,      7.7202,     33.3402,      8.0000,  

In [1109]:
vals,indices = t_indep.max(dim=0)
t_indep = t_indep / vals

In [1110]:
t_indep*coeffs

tensor([[-0.0070,  0.1443,  0.1673,  0.1378, -0.4768, -0.0260, -0.2365, -0.2289],
        [-0.0058,  0.1173,  0.1664,  0.2664, -0.4768, -0.0246, -0.2202, -0.2114],
        [-0.0016,  0.0555,  0.0634, -0.0551, -0.1589, -0.0139, -0.1121, -0.0459],
        [-0.0079,  0.1399,  0.1475,  0.1882, -0.4768, -0.0265, -0.2172, -0.2260],
        [-0.0073,  0.1828, -0.1692,  0.2225, -0.2119, -0.0284, -0.3087, -0.1703],
        [-0.0043,  0.1045,  0.0028, -0.2339, -0.2649, -0.0233, -0.2109, -0.0724],
        [-0.0026,  0.0717,  0.0963,  0.1850, -0.4768, -0.0177, -0.1450, -0.1586],
        ...,
        [-0.0015,  0.0698,  0.0402, -0.0511, -0.1589, -0.0135, -0.1407, -0.0000],
        [-0.0031,  0.0790, -0.0467,  0.3071, -0.4238, -0.0202, -0.1597, -0.0000],
        [-0.0024,  0.0770, -0.1106,  0.0097, -0.3178, -0.0172, -0.1557, -0.0000],
        [-0.0049,  0.1348,  0.0700,  0.0237, -0.2119, -0.0255, -0.2658, -0.0000],
        [-0.0035,  0.0861, -0.1522,  0.0608, -0.3178, -0.0217, -0.1740, -0.0000],
   

In [1111]:
t_indep = t_indep / vals

In [1112]:
independent_variables = pd.DataFrame(numerical_values)

In [1113]:
from fastai.data.transforms import RandomSplitter
trn_split,val_split=RandomSplitter(seed=42)(independent_variables)

In [1114]:
trn_indep,val_indep = t_indep[trn_split],t_indep[val_split]
trn_dep,val_dep = t_dep[trn_split],t_dep[val_split]
#len(trn_indep),len(val_indep)
#dropout, svd, latent space

In [1169]:

torch.manual_seed(443)

def calc_preds(coeffs, indeps): return (coeffs * indeps).sum(axis=1)
def calc_preds(coeffs, indeps): return torch.sigmoid((indeps*coeffs).sum(axis=1))
#def calc_preds(coeffs, indeps): return (indeps*coeffs).sum(axis=1)
#def calc_preds(coeffs, indeps): return torch.sigmoid(indeps@coeffs)

def calc_loss(coeffs, indeps, deps): return torch.abs(calc_preds(coeffs, indeps)-deps).mean()
def init_coeffs(): return (torch.rand(n_coeff)-0.5).requires_grad_()
def update_coeffs(coeffs, lr): 
    coeffs.sub_(coeffs.grad * lr)
    coeffs.grad.zero_()

def one_epoch(coeffs, lr):
    loss = calc_loss(coeffs, trn_indep, trn_dep)
    loss.backward()
    with torch.no_grad(): update_coeffs(coeffs, lr)
    print(f"{loss:.3f}", end="; ")
    
m = torch.nn.Softmax(dim=1)
input = torch.randn(2, 3)
one = m(input)
#print(one, input)

# def init_coeffs(): 
#     hiddens = [10, 10] # <-- set this to the size of each hidden layer you want_ 
#     sizes = [n_coeff] + hiddens + [1] 
#     n = len(sizes) 
#     layers = [(torch.rand(sizes[i], sizes[i+1])-0.3)/sizes[i+1]  for i in range(n-1)] 
#     consts = [(torch.rand(1)[0]-0.5)*0.1 for i in range(n-1)] 
#     for l in layers+consts:
#         l.requires_grad_() 
#     return layers,consts

# import torch.nn.functional as F 
# def calc_preds(coeffs, indeps): 
#     layers,consts = coeffs 
#     n = len(layers) 
#     res = indeps 
#     for i,l in enumerate(layers): 
#         res = res@l + consts[i] 
#         if i!=n-1: res = F.relu(res) 
#     return torch.sigmoid(res)

# def update_coeffs(coeffs, lr): 
#     layers, consts = coeffs 
#     for layer in layers+consts: 
#         layer.sub_(layer.grad * lr) 
#         layer.grad.zero_()
        
def train_model(epochs=30, lr=4):
    coeffs = init_coeffs()
    for i in range(epochs): 
        one_epoch(coeffs, lr=lr)
#         print(coeffs)
#         coeffs = m(coeffs)
    return coeffs

coeffs = train_model(33, lr=.02)

indep_cols = ['nCount_RNA' ,                
'nFeature_RNA'    ,             
'tsne1'     ,                 
'tsne2'  ,                   
'cluster'    ,                  
'nCount_SCT'   ,              
'nFeature_SCT',
'geneExpressionCount'
]    
def show_coeffs(): return dict(zip(indep_cols, coeffs.requires_grad_(False)))
print(show_coeffs())
len([prob for prob in calc_preds(coeffs, t_indep) if prob > .5]) # should be 5000

0.532; 0.532; 0.532; 0.532; 0.532; 0.532; 0.532; 0.532; 0.532; 0.532; 0.532; 0.532; 0.532; 0.532; 0.532; 0.532; 0.532; 0.532; 0.532; 0.532; 0.532; 0.532; 0.532; 0.532; 0.532; 0.532; 0.532; 0.532; 0.532; 0.532; 0.532; 0.532; 0.532; {'nCount_RNA': tensor(0.3870), 'nFeature_RNA': tensor(0.1283), 'tsne1': tensor(-0.0261), 'tsne2': tensor(-0.0653), 'cluster': tensor(-0.0117), 'nCount_SCT': tensor(0.1363), 'nFeature_SCT': tensor(-0.3976), 'geneExpressionCount': tensor(-0.2811)}


5686

In [1168]:
[prob.item() for prob in calc_preds(coeffs, t_indep)]

[0.4509545564651489,
 0.4509437382221222,
 0.4839145243167877,
 0.45091331005096436,
 0.47732439637184143,
 0.4733034670352936,
 0.45133858919143677,
 0.4733440577983856,
 0.45113489031791687,
 0.4627237021923065,
 0.446952760219574,
 0.48351556062698364,
 0.4733254611492157,
 0.47360503673553467,
 0.47823014855384827,
 0.4786529839038849,
 0.45100000500679016,
 0.4467613399028778,
 0.4732779264450073,
 0.4735549986362457,
 0.45134130120277405,
 0.4730464220046997,
 0.4786413013935089,
 0.4469107687473297,
 0.47341179847717285,
 0.4467383921146393,
 0.4732570946216583,
 0.4516659677028656,
 0.47324123978614807,
 0.47862011194229126,
 0.48333826661109924,
 0.4734020233154297,
 0.45077699422836304,
 0.4836588501930237,
 0.47327563166618347,
 0.4732299745082855,
 0.4732414484024048,
 0.4838330149650574,
 0.47824567556381226,
 0.4732862114906311,
 0.4508187472820282,
 0.45140066742897034,
 0.4937223792076111,
 0.44692304730415344,
 0.4783414900302887,
 0.4781681001186371,
 0.47322478890419

In [1161]:
coeffs

tensor([ 0.3870,  0.1283, -0.0261, -0.0653, -0.0117,  0.1363, -0.3976, -0.2811])

In [1160]:
def acc(coeffs): return (val_dep.bool()==(calc_preds(coeffs, val_indep)>0.1)).float().mean()

acc(coeffs) 

tensor(0.0020)

In [1057]:

# !pip install --quiet hyperopt
# !pip install --quiet "ray[tune]"
# !pip install --quiet scvi-colab
# from scvi_colab import install

# install()


# import ray
# import scanpy as sc
# import scvi
# from ray import tune
# from scvi import autotune
# model_cls = scvi.model.SCVI
# model_cls.setup_anndata(adata)
# scvi_tuner = autotune.ModelTuner(model_cls)


In [970]:
def benchmarkResults():
    pass

In [852]:
import scgen
adata.obs.rename({"label": "condition"}, axis=1, inplace=True)
adata.obs["condition"].replace({"ctrl": "control", "stim": "stimulated"}, inplace=True)




# https://genomebiology.biomedcentral.com/articles/10.1186/s13059-022-02605-1

# adata_t = adata[
#     ~(
#         (adata.obs["cell_type"] == "CD4 T cells")
#         & (adata.obs["condition"] == "stimulated")
#     )
# ].copy()

# cd4t_stim = adata[
#     (
#         (adata.obs["cell_type"] == "CD4 T cells")
#         & (adata.obs["condition"] == "stimulated")
#     )
# ].copy()








# scgen.SCGEN.setup_anndata(adata_t, batch_key="condition", labels_key="cell_type")


In [807]:
# model = scgen.SCGEN(adata_t, n_hidden=800, n_latent=100, n_layers=2)

In [642]:
# model.train(
#     max_epochs=1, batch_size=32, early_stopping=True, early_stopping_patience=25
# )

In [643]:
# adata_t.obsm["scgen"] = model.get_latent_representation()

In [644]:
# sc.pp.neighbors(adata_t, use_rep="scgen")
# sc.tl.umap(adata)

In [645]:
# adata_t

In [646]:
# pred, delta = model.predict(
#     ctrl_key="control", stim_key="stimulated", celltype_to_predict="CD4 T cells"
# )

# # we annotate the predicted cells to distinguish them later from ground truth cells.
# pred.obs["condition"] = "prediction"
# print(pred)

In [647]:
# pred.obs

In [648]:

# train = adata
# ctrl_adata = adata[
#     ((adata.obs["cell_type"] == "CD4 T cells") & (adata.obs["condition"] == "control"))
# ]
# stim_adata = train[((train.obs['cell_type'] == 'CD4T') & (train.obs['condition'] == 'stimulated'))]

# # concatenate pred, control and real CD4 T cells in to one object
# eval_adata = ctrl_adata.concatenate(cd4t_stim, pred)

In [649]:
# sc.tl.pca(eval_adata)
# sc.pl.pca(eval_adata, color="condition", frameon=False)

In [650]:
# cd4t_adata = adata[adata.obs["cell_type"] == "CD4 T cells"]

In [651]:
# sc.tl.rank_genes_groups(cd4t_adata, groupby="condition", method="wilcoxon")
# diff_genes = cd4t_adata.uns["rank_genes_groups"]["names"]["stimulated"]
# diff_genes

In [ ]:
# r2_value = model.reg_mean_plot(
#     eval_adata,
#     axis_keys={"x": "predicted stimulated", "y": "stimulated"},
#     gene_list=diff_genes[:10],
#     top_100_genes=diff_genes,
#     labels={"x": "predicted", "y": "ground truth"},
#     show=True,
#     legend=False,
# )

In [ ]:
# sc.pl.violin(eval_adata, keys="ISG15", groupby="condition")

In [ ]:
# import pertpy as pt
# import muon as mu
# import scanpy as sc

In [ ]:
# mdata = pt.dt.papalexi_2021()
# for col in mdata.obs: print(col)

In [ ]:
# sc.pp.normalize_total(mdata["rna"])
# sc.pp.log1p(mdata["rna"])
# sc.pp.highly_variable_genes(mdata["rna"], subset=True)

In [ ]:
# mu.prot.pp.clr(mdata["adt"])

In [ ]:
# sc.pp.pca(mdata["rna"])

In [ ]:
# We calculate neighbors with the cosine distance similarly to the original Seurat implementation
# sc.pp.neighbors(mdata["rna"], metric="cosine")

In [ ]:
# sc.tl.umap(mdata["rna"])

In [ ]:
# sc.pl.umap(mdata["rna"], color=["replicate", "Phase", "perturbation"])

In [ ]:
# ms = pt.tl.Mixscape()

# ms.perturbation_signature(
#     mdata["rna"],
#     pert_key="perturbation",
#     control="NT",
#     split_by="replicate",
#     n_neighbors=20,
# )

In [ ]:
# We create a copy of the object to recalculate the PCA.
# Alternatively we could replace the X of the RNA part of our MuData object with the `X_pert` layer.
# adata_pert = mdata["rna"].copy()
# adata_pert.X = adata_pert.layers["X_pert"]
# sc.pp.pca(adata_pert)
# sc.pp.neighbors(adata_pert, metric="cosine")
# sc.tl.umap(adata_pert)
# sc.pl.umap(adata_pert, color=["replicate", "Phase", "perturbation"])

In [ ]:
# ms.mixscape(adata=mdata["rna"], control="NT", labels="gene_target", layer="X_pert")

In [ ]:
# mdata["rna"].obs

In [ ]:
# pt.pl.ms.perturbscore(
#     adata=mdata["rna"], labels="gene_target", target_gene="IFNGR2", color="orange"
# )

In [ ]:
# sc.settings.set_figure_params(figsize=(10, 10))
# pt.pl.ms.violin(
#     adata=mdata["rna"],
#     target_gene_idents=["NT", "IFNGR2 NP", "IFNGR2 KO"],
#     groupby="mixscape_class",
# )

In [ ]:
# pt.pl.ms.heatmap(
#     adata=mdata["rna"],
#     labels="gene_target",
#     target_gene="IFNGR2",
#     layer="X_pert",
#     control="NT",
# )

In [ ]:
# mdata["adt"].obs["mixscape_class_global"] = mdata["rna"].obs["mixscape_class_global"]
# pt.pl.ms.violin(
#     adata=mdata["adt"],
#     target_gene_idents=["NT", "JAK2", "STAT1", "IFNGR1", "IFNGR2", "IRF1"],
#     keys="PDL1",
#     groupby="gene_target",
#     hue="mixscape_class_global",
# )

In [ ]:
# ms.lda(adata=mdata["rna"], labels="gene_target", layer="X_pert")

In [ ]:
# pt.pl.ms.lda(adata=mdata["rna"])
#https://zenodo.org/record/7058382
# folders = '/home/awahab/llm-testing/data_sets/'
# fp1 = 'SC3_v3_NextGem_DI_CRISPR_A549_5K_Multiplex_count_raw_feature_bc_matrix.h5'
# fp2 = 'SC3_v3_NextGem_DI_CRISPR_A549_5K_Multiplex_count_raw_molecule_info.h5'


# import h5py
# import anndata

# # Read the .h5 File
# def explore_h5py_group(group, indent=0):
#     """Recursively print the contents of an h5py group/dataset."""
#     print(group.data.name)
#     items = sorted(group.items())
#     for name, item in items:
#         if isinstance(item, h5py.Dataset):  # Check if item is a dataset
#             print("  " * indent + f"Dataset: {name} (Shape: {item.shape}, Dtype: {item.dtype})")
#         elif isinstance(item, h5py.Group):  # Check if item is a group
#             print("  " * indent + f"Group: {name}")
#             explore_h5py_group(item, indent + 1)  # Recursive call to explore subgroups

# # Open your HDF5 file
# with h5py.File(folders + fp1, 'r') as f:
#     explore_h5py_group(f)